In [3]:
cd ../

/project/bengali/bengali-2020


In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/folds.csv')
val_df = df.query('folds==0').copy()
root = 'results/062_srx50_plane_focal_cosine_ssr_msd_resize224_fold0'

val_df['g_pred'] = np.argmax(np.load(f'{root}/valid_preds_g.npy'),axis=1)
val_df['v_pred'] = np.argmax(np.load(f'{root}/valid_preds_v.npy'),axis=1)
val_df['c_pred'] = np.argmax(np.load(f'{root}/valid_preds_c.npy'),axis=1)

In [5]:
val_df['g_score'] = 1
val_df['v_score'] = 1
val_df['c_score'] = 1
val_df.loc[val_df['grapheme_root'] != val_df['g_pred'], 'g_score'] = 0
val_df.loc[val_df['vowel_diacritic'] != val_df['v_pred'], 'v_score'] = 0
val_df.loc[val_df['consonant_diacritic'] != val_df['c_pred'], 'c_score'] = 0

a_true = val_df[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values
a_pred = val_df[['g_pred', 'v_pred', 'c_pred']].values
val_df['a_score'] = np.all(a_true == a_pred, axis=1).astype(int)

In [6]:
val_df.mean(axis=0)

Unnamed: 0             1.002388e+05
original_index         1.002388e+05
grapheme_root          8.640953e+01
vowel_diacritic        3.364452e+00
consonant_diacritic    1.282924e+00
folds                  0.000000e+00
combined                        inf
unique_label           6.458664e+02
g_pred                 8.653198e+01
v_pred                 3.359432e+00
c_pred                 1.280862e+00
g_score                9.636219e-01
v_score                9.899612e-01
c_score                9.885200e-01
a_score                9.497316e-01
dtype: float64

In [7]:
g_result = {}
for c, _df in val_df.groupby('grapheme_root'):
    g_result[c] = _df.g_score.mean()
    
v_result = {}
for c, _df in val_df.groupby('vowel_diacritic'):
    v_result[c] = _df.v_score.mean()
    
c_result = {}
for c, _df in val_df.groupby('consonant_diacritic'):
    c_result[c] = _df.c_score.mean()
    
all_result = {}
for c, _df in val_df.groupby('combined'):
    all_result[c] = _df.a_score.mean()

In [8]:
pd.Series(g_result).sort_values().head(20)

60     0.674419
61     0.721739
62     0.781609
84     0.804348
162    0.806452
68     0.850394
154    0.863946
63     0.866667
30     0.875000
54     0.882353
145    0.890625
140    0.892655
122    0.896328
102    0.896552
120    0.896694
32     0.898230
156    0.905063
85     0.908425
144    0.910569
137    0.911504
dtype: float64

In [9]:
pd.Series(v_result).sort_values()

6     0.963387
5     0.971624
8     0.979259
4     0.985625
9     0.987585
10    0.989101
0     0.989158
1     0.991329
3     0.992175
7     0.992744
2     0.999039
dtype: float64

In [10]:
pd.Series(c_result).sort_values()

3    0.941176
6    0.967153
1    0.978567
2    0.979566
5    0.983849
4    0.984209
0    0.992810
dtype: float64

In [11]:
pd.Series(all_result).sort_values().head(20)

61_2_0     0.500000
60_4_0     0.631579
62_7_0     0.647059
64_4_5     0.666667
60_1_0     0.681818
61_0_0     0.700000
61_7_0     0.703704
53_2_5     0.714286
83_1_0     0.718750
79_7_0     0.718750
162_1_0    0.729730
53_9_0     0.730769
60_0_0     0.730769
68_3_0     0.730769
120_0_0    0.735294
122_9_0    0.736842
85_7_0     0.739130
62_0_0     0.742857
30_7_0     0.750000
154_4_0    0.750000
dtype: float64

In [12]:
val_df.query('a_score==0').g_pred.value_counts()

115    65
148    59
147    58
23     53
153    48
       ..
166     1
27      1
47      1
45      1
112     1
Name: g_pred, Length: 152, dtype: int64

In [13]:
val_df.query('a_score==0').grapheme_root.value_counts().head(20)

113    62
53     60
64     56
23     52
122    52
79     51
115    45
81     42
62     39
96     38
71     38
13     36
107    36
56     33
61     32
141    32
72     32
133    32
43     31
29     31
Name: grapheme_root, dtype: int64

In [47]:
val_df.query('combined=="61_2_0"&a_score==0')

,Unnamed: 0,original_index,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,folds,combined,unique_label,g_pred,v_pred,c_pred,g_score,v_score,c_score,a_score
7851,7851,7851,Train_7851,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0
8213,8213,8213,Train_8213,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0
16929,16929,16929,Train_16929,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0
33939,33939,33939,Train_33939,61,2,0,0.0,61_2_0,935,59,2,0,0,1,1,0
37475,37475,37475,Train_37475,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0
61581,61581,61581,Train_61581,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0
65969,65969,65969,Train_65969,61,2,0,0.0,61_2_0,935,142,2,0,0,1,1,0
77144,77144,77144,Train_77144,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0
88164,88164,88164,Train_88164,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0
111364,111364,111364,Train_111364,61,2,0,0.0,61_2_0,935,84,2,0,0,1,1,0


In [42]:
train_df = pd.read_csv('../input/bengaliai-cv19/train.csv')

In [44]:

train_df['combined'] = train_df['grapheme_root'].astype(str) + '_' + train_df['vowel_diacritic'].astype(str) + '_' + train_df['consonant_diacritic'].astype(str)

In [59]:
top = ['61_2_0', '60_4_0', '62_7_0', '64_4_5', '60_1_0', '61_0_0', '61_7_0',]
#        '53_2_5', '83_1_0', '79_7_0', '162_1_0', '53_9_0', '60_0_0', '68_3_0',
#        '120_0_0', '122_9_0', '85_7_0', '62_0_0', '30_7_0', '154_4_0']
top_df = train_df[train_df.combined.map(lambda x: x in top)]

In [62]:
top_df[['grapheme', 'combined']].drop_duplicates()

,grapheme,combined
21,ণ্টু,60_4_0
202,ণ্ডে,62_7_0
730,ণ্ঠে,61_7_0
754,ত্রু,64_4_5
1784,ণ্ঠ,61_0_0
2996,ণ্টা,60_1_0
3032,ণ্ঠি,61_2_0


In [1]:
from sklearn.metrics import f1_score

In [15]:
f1_score(val_df.grapheme_root, val_df.g_pred, average='macro',)

0.9584198523766987